In [1]:

import numpy as np
import scipy.sparse

def create_adj(filename,N):
    adj =  scipy.sparse.dok_matrix((N,N))
    with open(filename, 'r') as myfile:
        for line in myfile:
            try:
                m,n =  line.strip().split()
                adj[int(m),int(n)] = 1
#                 print(adj[m,n])
            except:
                pass
    return adj

A = create_adj('matrix.txt', 8)
print(A)

  (0, 7)	1.0
  (1, 0)	1.0
  (3, 0)	1.0
  (3, 2)	1.0
  (3, 6)	1.0
  (4, 0)	1.0
  (4, 5)	1.0
  (4, 6)	1.0
  (5, 0)	1.0
  (5, 6)	1.0
  (6, 0)	1.0
  (7, 0)	1.0


In [2]:
import numpy as np
import scipy.linalg

def K_givenA(A):
    n,n = np.shape(A)
    B = np.empty((n,n))
    diagonals = np.empty(n)
    for i in range(n):
        if any(A[i,:])==0:
            B[i,:]=1
        else:
            B[i,:]=A[i,:]
        diagonals[i] = np.sum(B, axis=1)[i]
    D = np.eye(n)*diagonals
    Dinv=scipy.linalg.inv(D)
    K = (Dinv@B).T
    return K    

A = np.array([0,0,0,0,0,0,0,1, 1,0,0,0,0,0,0,0, 0,0,0,0,0,0,0,0, 1,0,1,0,0,0,1,0, 1,0,0,0,0,1,1,0, 1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0, 1,0,0,0,0,0,0,0]).reshape(8,8)
K_givenA(A)
  


array([[0.        , 1.        , 0.125     , 0.33333333, 0.33333333,
        0.5       , 1.        , 1.        ],
       [0.        , 0.        , 0.125     , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.33333333, 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.        , 0.33333333,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.125     , 0.33333333, 0.33333333,
        0.5       , 0.        , 0.        ],
       [1.        , 0.        , 0.125     , 0.        , 0.        ,
        0.        , 0.        , 0.        ]])

In [3]:
import numpy as np
import scipy.linalg

N = None
d = 0.85
tol = 1e-5
iter_max=100

def K_givenA(A):
    n,n = np.shape(A)
    B = np.empty((n,n))
    diagonals = np.empty(n)
    for i in range(n):
        if any(A[i,:])==0:
            B[i,:]=1
        else:
            B[i,:]=A[i,:]
        diagonals[i] = np.sum(B, axis=1)[i]
    D = np.eye(n)*diagonals
    Dinv=scipy.linalg.inv(D)
    K = (Dinv@B).T
    return K  

def iterative(A,N,d,tol):
    n,n = np.shape(A)
    pt = np.random.random(n)
    ones = np.ones(n)
    K=K_givenA(A)
    iter = 0
    dist=5
    while (iter<iter_max)& (dist>tol):
        pt1 = d*np.asarray(np.matrix(K)@np.matrix(pt).T).reshape(-1) + ((1-d)/n)*ones
        dist = scipy.linalg.norm(pt1-pt)
        pt = pt1
        iter +=1
    return pt1

A  = np.array([0,0,0,0,0,0,0,1, 1,0,0,0,0,0,0,0, 0,0,0,0,0,0,0,0, 1,0,1,0,0,0,1,0, 
               1,0,0,0,0,1,1,0, 1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0, 1,0,0,0,0,0,0,0]).reshape(8,8)
iterative(A,N,d,tol)

array([0.43869933, 0.02171029, 0.02786154, 0.02171029, 0.02171029,
       0.02786154, 0.04585394, 0.39461103])

In [5]:
import numpy as np
import scipy.linalg

N= None
d=0.85
tole=1e-15
iter_max=100

def eigen(A,N):
    m,n = np.shape(A)
    x=[None]*(N+1)
    x[0]=np.random.random(n)
    x[0]=x[0]
    x[0]=x[0]/scipy.linalg.norm(x[0])
    for k in range(N):
        matrix=(np.matrix(A)@(np.matrix(x[k]).T)).T
        x[k+1]=np.asarray(matrix).reshape(-1)
        x[k+1]=x[k+1]/scipy.linalg.norm(x[k+1])
    eigenvalue=np.float((np.matrix(x[N]))@(np.matrix(A))@(np.matrix(x[N]).T))
    return eigenvalue,x[N]


def iterationmatrix(A,N,d,tol):
    n,n = np.shape(A)
    K = K_givenA(A)
    E = np.ones((n,n))
    B = d*K + ((1-d)/n)*E
    p = scipy.linalg.solve(np.eye(n) - d * K, ((1 - d) / n) * np.ones(n))
    return p

A  = np.array([0,0,0,0,0,0,0,1, 1,0,0,0,0,0,0,0, 0,0,0,0,0,0,0,0, 1,0,1,0,0,0,1,0, 
               1,0,0,0,0,1,1,0, 1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0, 1,0,0,0,0,0,0,0]).reshape(8,8)
iterationmatrix(A,N,d,tol)
        
    

array([0.43869288, 0.02171029, 0.02786154, 0.02171029, 0.02171029,
       0.02786154, 0.04585394, 0.39459924])

In [4]:
import numpy as np
import scipy.linalg
import pandas as pd

iter_max=100

def K_givenA(A):
    n,n = np.shape(A)
    B = np.empty((n,n))
    diagonals = np.empty(n)
    for i in range(n):
        if any(A[i,:])==0:
            B[i,:]=1
        else:
            B[i,:]=A[i,:]
        diagonals[i] = np.sum(B, axis=1)[i]
    D = np.eye(n)*diagonals
    Dinv=scipy.linalg.inv(D)
    K = (Dinv@B).T
    return K  

def iterative(A,N,d,tol):
    n,n = np.shape(A)
    pt = np.random.random(n)
    ones = np.ones(n)
    K=K_givenA(A)
    iter = 0
    dist=5
    while (iter<iter_max)& (dist>tol):
        pt1 = d*np.asarray(np.matrix(K)@np.matrix(pt).T).reshape(-1) + ((1-d)/n)*ones
        dist = scipy.linalg.norm(pt1-pt)
        pt = pt1
        iter +=1
    return pt1


ncaa = pd.read_csv('ncaa2013.csv', header = 0).as_matrix()
m, n = ncaa.shape
teams, idx = np.unique(ncaa, return_inverse = True)
idx = idx.reshape((m, n))

t = len(teams)
Adj = np.zeros((t,t))

for i in range(m):
    Adj[idx[i, 1], idx[i, 0]] = 1

ranks = iterative(Adj,N=None, d = 0.7, tol =1e-5)
np.argsort(ranks.T)
ranks = teams[np.argsort(ranks.T)].T[::-1]

print('The top 5 ranked teams are, in order:')
for i in range(5):
    print(ranks[i][0])

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


The top 5 ranked teams are, in order:
D
B
L
I
I
